# PyRevealed: Real Data Analysis

This notebook tests the package on real experimental data from the Prest project.

**Data Source**: [Prest budgetary dataset](https://github.com/prestsoftware/prest) - Laboratory experiments with real subjects making choices under budget constraints.

In [ ]:
import pandas as pd
import numpy as np

from pyrevealed import (
    BehaviorLog,
    validate_consistency,
    compute_integrity_score,
    compute_confusion_metric,
    BehavioralAuditor,
)

print("Imports successful!")

## Load Real Data from Prest

The Prest project provides budgetary choice data with Price1-6 and Demand1-6 columns.

In [ ]:
# Load budgetary dataset from Prest GitHub
url = "https://raw.githubusercontent.com/prestsoftware/prest/master/docs/src/_static/examples/budgetary.csv"

try:
    budgetary = pd.read_csv(url)
    print(f"Loaded dataset: {budgetary.shape}")
    print(f"Columns: {list(budgetary.columns)}")
    print(f"\nSubjects: {budgetary['Subject'].unique()}")
except Exception as e:
    print(f"Failed to load data: {e}")
    print("\nCreating synthetic fallback data...")
    # Fallback synthetic data
    budgetary = None

In [ ]:
if budgetary is not None:
    print("Sample rows:")
    display(budgetary.head())

## Analyze Each Subject

We'll analyze each subject's behavioral consistency.

In [ ]:
if budgetary is not None:
    price_cols = [f"Price{i}" for i in range(1, 7)]
    demand_cols = [f"Demand{i}" for i in range(1, 7)]
    
    results = []
    
    for subject in budgetary['Subject'].unique():
        subject_data = budgetary[budgetary['Subject'] == subject]
        prices = subject_data[price_cols].values
        quantities = subject_data[demand_cols].values
        
        # Filter out zero-price columns (some subjects have fewer goods)
        valid_cols = (prices > 0).any(axis=0)
        prices = prices[:, valid_cols]
        quantities = quantities[:, valid_cols]
        
        # Create BehaviorLog using tech-friendly API
        log = BehaviorLog(
            cost_vectors=prices, 
            action_vectors=quantities,
            user_id=str(subject)
        )
        
        # Run analyses
        garp_result = validate_consistency(log)
        integrity_result = compute_integrity_score(log)
        confusion_result = compute_confusion_metric(log)
        
        results.append({
            'Subject': subject,
            'Observations': log.num_records,
            'Goods': log.num_features,
            'GARP_Consistent': garp_result.is_consistent,
            'Violations': len(garp_result.violations),
            'Integrity': integrity_result.efficiency_index,
            'Confusion': confusion_result.mpi_value,
        })
    
    results_df = pd.DataFrame(results)
    print("Analysis Results:")
    display(results_df)
else:
    print("No data available - using synthetic fallback")

## Summary Statistics

In [ ]:
if budgetary is not None and len(results) > 0:
    print(f"Total subjects: {len(results_df)}")
    print(f"GARP consistent: {results_df['GARP_Consistent'].sum()} ({100*results_df['GARP_Consistent'].mean():.1f}%)")
    print(f"\nIntegrity Score (AEI):")
    print(f"  Mean: {results_df['Integrity'].mean():.3f}")
    print(f"  Min:  {results_df['Integrity'].min():.3f}")
    print(f"  Max:  {results_df['Integrity'].max():.3f}")
    print(f"\nConfusion Score (MPI):")
    print(f"  Mean: {results_df['Confusion'].mean():.3f}")
    print(f"  Min:  {results_df['Confusion'].min():.3f}")
    print(f"  Max:  {results_df['Confusion'].max():.3f}")

## Using BehavioralAuditor for Batch Analysis

In [ ]:
if budgetary is not None:
    # Pick one subject for detailed audit
    subject = budgetary['Subject'].unique()[0]
    subject_data = budgetary[budgetary['Subject'] == subject]
    prices = subject_data[price_cols].values
    quantities = subject_data[demand_cols].values
    
    valid_cols = (prices > 0).any(axis=0)
    prices = prices[:, valid_cols]
    quantities = quantities[:, valid_cols]
    
    log = BehaviorLog(cost_vectors=prices, action_vectors=quantities)
    
    # Use high-level auditor
    auditor = BehavioralAuditor()
    report = auditor.full_audit(log)
    
    print(f"Subject {subject} Full Audit:")
    print(f"  Consistent: {report.is_consistent}")
    print(f"  Integrity:  {report.integrity_score:.3f}")
    print(f"  Confusion:  {report.confusion_score:.3f}")

## Fallback: Synthetic Real-World Style Data

If the remote data isn't available, create synthetic data mimicking real patterns.

In [ ]:
if budgetary is None:
    print("Creating synthetic grocery shopping data...")
    
    np.random.seed(42)
    
    # Simulate 10 weeks of grocery shopping with 5 categories
    # Categories: Produce, Dairy, Meat, Bakery, Snacks
    n_weeks = 10
    n_categories = 5
    
    # Base prices with some variation
    base_prices = np.array([3.0, 4.0, 8.0, 2.5, 3.5])
    prices = base_prices * (1 + 0.2 * np.random.randn(n_weeks, n_categories))
    prices = np.maximum(prices, 0.5)  # Ensure positive
    
    # Quantities inversely related to prices (rational behavior)
    budget = 50.0
    quantities = budget / prices + 0.5 * np.random.randn(n_weeks, n_categories)
    quantities = np.maximum(quantities, 0)  # Ensure non-negative
    
    log = BehaviorLog(
        cost_vectors=prices,
        action_vectors=quantities,
        user_id="synthetic_shopper"
    )
    
    print(f"Created log with {log.num_records} observations, {log.num_features} features")
    
    result = validate_consistency(log)
    print(f"\nGARP consistent: {result.is_consistent}")
    
    integrity = compute_integrity_score(log)
    print(f"Integrity score: {integrity.efficiency_index:.3f}")

## Summary

### Issues Found

1. **SSL Certificate Error**: On some macOS Python installations, loading remote data fails with 
   `CERTIFICATE_VERIFY_FAILED`. This is a system-level Python issue, not a PyRevealed bug.
   - Fix: Run `/Applications/Python X.X/Install Certificates.command` or use `certifi`
   
2. **No explicit batch processing API**: When analyzing many subjects, users must loop manually.
   A batch API like `auditor.audit_batch([log1, log2, ...])` would be convenient.

### What Worked

- Clean fallback to synthetic data when remote loading fails
- BehaviorLog works seamlessly with pandas-extracted numpy arrays
- Batch analysis loop pattern works correctly
- All metrics compute correctly for each subject
- Results can be collected into DataFrames for comparison